In [188]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [189]:
from bs4 import BeautifulSoup
from selenium import webdriver

In [190]:
# chrome driver 호출
browser = webdriver.Chrome('chromedriver.exe')

In [191]:
url = 'https://www.istarbucks.co.kr/store/store_map.do?disp=locale'

In [192]:
# url page로 이동
browser.get(url)

In [193]:
# '서울' 항목 클릭 => 서울 매장 목록으로 이동
seoul_bin = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(seoul_bin).click()

In [194]:
# '전체' 항목 클릭 => 스타벅스 서울지역 전체 목록페이지 
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
browser.find_element_by_css_selector(all_btn).click()

In [195]:
# 현재 상태의 페이지의 html source code를 저장
html = browser.page_source

In [196]:
# html code를 Beautifulsoup으로 파싱..
soup = BeautifulSoup(html, 'html.parser')

In [197]:
# 서울지역의 모든 스타벅스 매장 정보를 뽑아내기..

In [198]:
star_list = soup.select('li.quickResultLstCon')
len(star_list)

528

In [199]:
star_list[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong>역삼아레나빌딩  <img alt="" class="setStoreFavBtn mCS_img_loaded" data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N" src="//image.istarbucks.co.kr/common/img/store/icon_fav_off.png"/></strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

In [200]:
name = star_list[0].select('strong')[0].text.strip()
name

'역삼아레나빌딩'

In [201]:
lat = star_list[0]['data-lat'].strip()
lat

'37.501087'

In [202]:
long = star_list[0]['data-long'].strip()
long

'127.043069'

In [203]:
store_style = star_list[0].select('i')[0]['class'][0][4:]
store_style

'general'

In [204]:
address = str(star_list[0].select('p')[0]).split('<br/>')[0].split('>')[1]
address

'서울특별시 강남구 언주로 425 (역삼동)'

In [205]:
tel = str(star_list[0].select('p')[0]).split('<br/>')[1].split('<')[0]
tel

'1522-3232'

In [206]:
# 582개의 매장정보가 li tag로 저장되어있음 
# => 582 모든 매장의 name, lat, long, style, address, tel을 추출해서 list에 담기..

In [207]:
starbucks_list = []

for i in range(len(star_list)):
    name = star_list[i].select('strong')[0].text.strip()
    lat = star_list[i]['data-lat'].strip()
    long = star_list[i]['data-long'].strip()
    store_style = star_list[i].select('i')[0]['class'][0][4:]
    address = str(star_list[i].select('p')[0]).split('<br/>')[0].split('>')[1]
    tel = str(star_list[i].select('p')[0]).split('<br/>')[1].split('<')[0]
#     print(i, name, lat, long, address, tel)
#     print('-')
    starbucks_list.append([name, lat, long, store_style, address, tel])

In [208]:
len(starbucks_list)

528

In [209]:
starbucks_df = pd.DataFrame(starbucks_list)

In [210]:
starbucks_df.head()

,0,1,2,3,4,5
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [211]:
starbucks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 528 entries, 0 to 527
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       528 non-null    object
 1   1       528 non-null    object
 2   2       528 non-null    object
 3   3       528 non-null    object
 4   4       528 non-null    object
 5   5       528 non-null    object
dtypes: object(6)
memory usage: 24.9+ KB


In [212]:
starbucks_df.to_csv('./data/starbucks_list.csv')

In [213]:
# 위에서 스타벅스 매장 데이터를 추출..

In [214]:
# 1. 구별 시군구목록 데이터, 2. 구별 주민등록인구데이터, 3. 구별 사업체현황데이터..
# => 서울시 열린데이터광장에서 open api로 수집..
# => 구별 스타벅스 매장수와 구별 인구수 및 구별 사업체수를 비교..

In [215]:
# https://data.seoul.go.kr/together/guide/useGuide.do1
# 1. 서울시 열린데이터 광장 회원가입  , 'https://'
# 2. open api 인증키 발급

In [216]:
# 1. 시군구목록데이터 open api 요청..

In [220]:
import requests
import json

In [221]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [222]:
# 서울시 행정구역 시군구 정보 (좌표계: WGS1984)
# https://data.seoul.go.kr/dataList/OA-11677/S/1/datasetView.do

In [225]:
url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25'.format('7068476a49706f6f3736707a664d77')

In [226]:
res = requests.get(url)

In [227]:
result_dict = res.json()

In [228]:
result_data = result_dict['SdeTlSccoSigW']

In [229]:
result_data['list_total_count']

25

In [230]:
result_data['RESULT']['CODE']

'INFO-000'

In [231]:
result_data['row']

[{'OBJECTID': 1.0,
  'SIG_CD': '11320',
  'SIG_KOR_NM': '도봉구',
  'SIG_ENG_NM': 'Dobong-gu',
  'ESRI_PK': 0.0,
  'LAT': '37.6658609',
  'LNG': '127.0317674'},
 {'OBJECTID': 2.0,
  'SIG_CD': '11380',
  'SIG_KOR_NM': '은평구',
  'SIG_ENG_NM': 'Eunpyeong-gu',
  'ESRI_PK': 1.0,
  'LAT': '37.6176125',
  'LNG': '126.9227004'},
 {'OBJECTID': 3.0,
  'SIG_CD': '11230',
  'SIG_KOR_NM': '동대문구',
  'SIG_ENG_NM': 'Dongdaemun-gu',
  'ESRI_PK': 2.0,
  'LAT': '37.5838012',
  'LNG': '127.0507003'},
 {'OBJECTID': 4.0,
  'SIG_CD': '11590',
  'SIG_KOR_NM': '동작구',
  'SIG_ENG_NM': 'Dongjak-gu',
  'ESRI_PK': 3.0,
  'LAT': '37.4965037',
  'LNG': '126.9443073'},
 {'OBJECTID': 5.0,
  'SIG_CD': '11545',
  'SIG_KOR_NM': '금천구',
  'SIG_ENG_NM': 'Geumcheon-gu',
  'ESRI_PK': 4.0,
  'LAT': '37.4600969',
  'LNG': '126.9001546'},
 {'OBJECTID': 6.0,
  'SIG_CD': '11530',
  'SIG_KOR_NM': '구로구',
  'SIG_ENG_NM': 'Guro-gu',
  'ESRI_PK': 5.0,
  'LAT': '37.4954856',
  'LNG': '126.858121'},
 {'OBJECTID': 7.0,
  'SIG_CD': '11110',
  '

In [232]:
sample_df = pd.DataFrame(result_data['row'])
sample_df.head()

,OBJECTID,SIG_CD,SIG_KOR_NM,SIG_ENG_NM,ESRI_PK,LAT,LNG
0,1.0,11320,도봉구,Dobong-gu,0.0,37.6658609,127.0317674
1,2.0,11380,은평구,Eunpyeong-gu,1.0,37.6176125,126.9227004
2,3.0,11230,동대문구,Dongdaemun-gu,2.0,37.5838012,127.0507003
3,4.0,11590,동작구,Dongjak-gu,3.0,37.4965037,126.9443073
4,5.0,11545,금천구,Geumcheon-gu,4.0,37.4600969,126.9001546


In [236]:
auth_key = '7068476a49706f6f3736707a664d77'
# api로 서비스 데이터를 요청해서 데이터를 받아서 리턴하는 함수..
def seoul_open_api_data(url, service):
    data_list = None
    try:
        result_dict = requests.get(url).json()
        result_data = result_dict[service]
        code = result_data['RESULT']['CODE']
        if code == 'INFO-000':
            data_list = result_data['row']
    except:
        pass
    return data_list

In [237]:
auth_key = '7068476a49706f6f3736707a664d77'
url = 'http://openapi.seoul.go.kr:8088/{}/json/SdeTlSccoSigW/1/25'.format(auth_key)
service = 'SdeTlSccoSigW'

sgg_data_list = seoul_open_api_data(url, service)

In [238]:
sgg_data_list

[{'OBJECTID': 1.0,
  'SIG_CD': '11320',
  'SIG_KOR_NM': '도봉구',
  'SIG_ENG_NM': 'Dobong-gu',
  'ESRI_PK': 0.0,
  'LAT': '37.6658609',
  'LNG': '127.0317674'},
 {'OBJECTID': 2.0,
  'SIG_CD': '11380',
  'SIG_KOR_NM': '은평구',
  'SIG_ENG_NM': 'Eunpyeong-gu',
  'ESRI_PK': 1.0,
  'LAT': '37.6176125',
  'LNG': '126.9227004'},
 {'OBJECTID': 3.0,
  'SIG_CD': '11230',
  'SIG_KOR_NM': '동대문구',
  'SIG_ENG_NM': 'Dongdaemun-gu',
  'ESRI_PK': 2.0,
  'LAT': '37.5838012',
  'LNG': '127.0507003'},
 {'OBJECTID': 4.0,
  'SIG_CD': '11590',
  'SIG_KOR_NM': '동작구',
  'SIG_ENG_NM': 'Dongjak-gu',
  'ESRI_PK': 3.0,
  'LAT': '37.4965037',
  'LNG': '126.9443073'},
 {'OBJECTID': 5.0,
  'SIG_CD': '11545',
  'SIG_KOR_NM': '금천구',
  'SIG_ENG_NM': 'Geumcheon-gu',
  'ESRI_PK': 4.0,
  'LAT': '37.4600969',
  'LNG': '126.9001546'},
 {'OBJECTID': 6.0,
  'SIG_CD': '11530',
  'SIG_KOR_NM': '구로구',
  'SIG_ENG_NM': 'Guro-gu',
  'ESRI_PK': 5.0,
  'LAT': '37.4954856',
  'LNG': '126.858121'},
 {'OBJECTID': 7.0,
  'SIG_CD': '11110',
  '

In [239]:
sgg_df = pd.DataFrame(sgg_data_list)
sgg_df = sgg_df[['SIG_CD','SIG_KOR_NM','LAT','LNG']]

In [240]:
sgg_df.columns = ['시군구코드','시군구명','위도','경도']
sgg_df.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.6658609,127.0317674
1,11380,은평구,37.6176125,126.9227004
2,11230,동대문구,37.5838012,127.0507003
3,11590,동작구,37.4965037,126.9443073
4,11545,금천구,37.4600969,126.9001546


In [241]:
sgg_df.to_excel('./data/4_2_seoul_list.xlsx', index=False)

In [242]:
# 2. 구별주민등록 인구데이터 open api 요청..

In [243]:
auth_key = '7068476a49706f6f3736707a664d77'
url = 'http://openapi.seoul.go.kr:8088/{}/json/octastatapi419/1/26'.format(auth_key)
service = 'octastatapi419'

pop_data_list = seoul_open_api_data(url, service)
pop_data_list

[{'GIGAN': '\ufeff2010.1-4',
  'JACHIGU': '합계',
  'SEDAE': '4,121,193',
  'GYE_1': '10,464,171',
  'NAMJA_1': '5,181,117',
  'YEOJA_1': '5,283,054',
  'GYE_2': '10,208,670',
  'NAMJA_2': '5,059,504',
  'YEOJA_2': '5,149,166',
  'GYE_3': '255,501',
  'NAMJA_3': '121,613',
  'YEOJA_3': '133,888',
  'SEDAEDANGINGU': '2.48',
  'N_65SEISANGGORYEONGJA': '956,414'},
 {'GIGAN': '2010.1-4',
  'JACHIGU': '종로구',
  'SEDAE': '74,300',
  'GYE_1': '177,023',
  'NAMJA_1': '88,225',
  'YEOJA_1': '88,798',
  'GYE_2': '168,158',
  'NAMJA_2': '84,088',
  'YEOJA_2': '84,070',
  'GYE_3': '8,865',
  'NAMJA_3': '4,137',
  'YEOJA_3': '4,728',
  'SEDAEDANGINGU': '2.26',
  'N_65SEISANGGORYEONGJA': '20,925'},
 {'GIGAN': '2010.1-4',
  'JACHIGU': '중구',
  'SEDAE': '58,472',
  'GYE_1': '138,102',
  'NAMJA_1': '69,175',
  'YEOJA_1': '68,927',
  'GYE_2': '129,786',
  'NAMJA_2': '65,216',
  'YEOJA_2': '64,570',
  'GYE_3': '8,316',
  'NAMJA_3': '3,959',
  'YEOJA_3': '4,357',
  'SEDAEDANGINGU': '2.22',
  'N_65SEISANGGORYE

In [244]:
sgg_pop_df = pd.DataFrame(pop_data_list)
sgg_pop_df.head()

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,﻿2010.1-4,합계,"4,121,193","10,464,171","5,181,117","5,283,054","10,208,670","5,059,504","5,149,166","255,501","121,613","133,888",2.48,"956,414"
1,2010.1-4,종로구,"74,300","177,023","88,225","88,798","168,158","84,088","84,070","8,865","4,137","4,728",2.26,"20,925"
2,2010.1-4,중구,"58,472","138,102","69,175","68,927","129,786","65,216","64,570","8,316","3,959","4,357",2.22,"16,208"
3,2010.1-4,용산구,"107,407","250,844","123,295","127,549","238,366","116,764","121,602","12,478","6,531","5,947",2.22,"28,436"
4,2010.1-4,성동구,"126,942","316,113","158,228","157,885","308,144","154,389","153,755","7,969","3,839","4,130",2.43,"30,704"


In [245]:
sgg_pop_df_selected = sgg_pop_df[sgg_pop_df['JACHIGU'] != '합계']

In [246]:
sgg_pop_df_selected.head(2)

,GIGAN,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
1,2010.1-4,종로구,"74,300","177,023","88,225","88,798","168,158","84,088","84,070","8,865","4,137","4,728",2.26,"20,925"
2,2010.1-4,중구,"58,472","138,102","69,175","68,927","129,786","65,216","64,570","8,316","3,959","4,357",2.22,"16,208"


In [247]:
columns = ['JACHIGU','GYE_1']
sgg_pop_df_final = sgg_pop_df_selected[columns]

In [248]:
sgg_pop_df_final.head()

,JACHIGU,GYE_1
1,종로구,"177,023"
2,중구,"138,102"
3,용산구,"250,844"
4,성동구,"316,113"
5,광진구,"386,292"


In [249]:
sgg_pop_df_final.columns = ['시군구명','주민등록인구']
sgg_pop_df_final.head()

,시군구명,주민등록인구
1,종로구,"177,023"
2,중구,"138,102"
3,용산구,"250,844"
4,성동구,"316,113"
5,광진구,"386,292"


In [250]:
sgg_pop_df_final.to_excel('./data/4_3_sgg_pop.xlsx', index=False)

In [251]:
# 3. 서울시 동별 사업체현황 open api 요청..

In [254]:
auth_key = '7068476a49706f6f3736707a664d77'
url = 'http://openapi.seoul.go.kr:8088/{}/json/octastatapi104/1/450'.format(auth_key)
service = 'octastatapi104'

sgg_biz_list = seoul_open_api_data(url, service)

sgg_biz_df = pd.DataFrame(sgg_biz_list)
sgg_biz_df.head(5)

,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,SAEOPCHESU_2,JONGSAJASU_1,...,JONGSAJASU_4,SAEOPCHESU_6,JONGSAJASU_5,SAEOPCHESU_7,JONGSAJASU_6,SAEOPCHESU_8,JONGSAJASU_7,SAEOPCHESU_9,JONGSAJASU_8,SAEOPCHESU_10
0,2015,합계,합계,820658,272567,5108828,2887719,2221109,17,491,...,8713,463,6712,22695,360192,234494,912406,93642,279921,126785
1,2015,종로구,소계,40871,13472,269106,152766,116340,1,16,...,387,9,179,358,25269,17447,46970,1418,7841,6635
2,2015,종로구,사직동,3551,1092,51425,29781,21644,1,16,...,2,2,32,38,4158,563,4074,280,2281,856
3,2015,종로구,삼청동,738,354,4736,2374,2362,-,-,...,-,-,-,6,15,273,864,20,84,261
4,2015,종로구,부암동,540,224,3635,1714,1921,-,-,...,-,-,-,22,53,104,523,33,37,125


In [259]:
sgg_biz_df_selected = sgg_biz_df[sgg_biz_df['DONG'] == '소계']

In [260]:
sgg_biz_df_selected.head(2)

,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,SAEOPCHESU_2,JONGSAJASU_1,...,JONGSAJASU_4,SAEOPCHESU_6,JONGSAJASU_5,SAEOPCHESU_7,JONGSAJASU_6,SAEOPCHESU_8,JONGSAJASU_7,SAEOPCHESU_9,JONGSAJASU_8,SAEOPCHESU_10
1,2015,종로구,소계,40871,13472,269106,152766,116340,1,16,...,387,9,179,358,25269,17447,46970,1418,7841,6635
19,2015,중구,소계,66190,24249,423808,234337,189471,-,-,...,1048,8,159,451,7364,34785,100132,1666,16214,6142


In [261]:
sgg_biz_final = sgg_biz_df_selected[['JACHIGU','GYE','SAEOPCHESU_1']]
sgg_biz_final.columns = ['시군구명','종사자수','사업체수']
sgg_biz_final.head()

,시군구명,종사자수,사업체수
1,종로구,269106,40871
19,중구,423808,66190
35,용산구,133446,21178
52,성동구,162019,26130
70,광진구,123689,24531


In [262]:
sgg_biz_df_final = sgg_biz_final.reset_index(drop=True)
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,269106,40871
1,중구,423808,66190
2,용산구,133446,21178
3,성동구,162019,26130
4,광진구,123689,24531


In [263]:
sgg_biz_df_final.to_excel('./data/4_3_sgg_biz_.xlsx', index=False)

In [264]:
### 여기까지 데이터 수집 저장 및 기초 전처리..

In [268]:
seoul_star = pd.read_excel('./data/4_1_seoul_starbucks_list.xlsx')
seoul_star.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [269]:
# '구' 정보를 담는 '시군구명' 칼럼 생성..

In [270]:
seoul_star['주소'].str.split(' ', expand=True)[1]

0      강남구
1      강남구
2      강남구
3      강남구
4      강남구
      ... 
523    중랑구
524    중랑구
525    중랑구
526    중랑구
527    중랑구
Name: 1, Length: 528, dtype: object

In [271]:
seoul_star['시군구명'] = seoul_star['주소'].str.split(' ', expand=True)[1]

In [272]:
seoul_star.head()

,매장명,위도,경도,매장타입,주소,전화번호,시군구명
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232,강남구
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232,강남구
2,신사역성일빌딩,37.514132,127.020563,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232,강남구
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,강남구
4,스탈릿대치R,37.494668,127.062583,reserve,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,강남구


In [273]:
seoul_star.to_excel('./data/4_4_seoul_starbucks_list.xlsx', index=False)

In [277]:
# seoul_sgg df에 '스타벅스매장수' 칼럼을 추가
seoul_sgg = pd.read_excel('./data/4_2_seoul_sgg_list.xlsx')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도
0,11320,도봉구,37.665861,127.031767
1,11380,은평구,37.617612,126.922700
2,11230,동대문구,37.583801,127.050700
3,11590,동작구,37.496504,126.944307
4,11545,금천구,37.460097,126.900155


In [278]:
# 구별매장수..
seoul_starbucks = pd.read_excel('./data/4_4_seoul_starbucks_list.xlsx')
s_count = seoul_starbucks.groupby('시군구명')['매장명'].count().to_frame()
s_count.head()

,매장명
시군구명,
강남구,81
강동구,15
강북구,5
강서구,16
관악구,11


In [279]:
seoul_sgg = pd.merge(seoul_sgg, s_count, left_on='시군구명', right_index=True)
seoul_sgg = seoul_sgg.rename(columns={'매장명':'스타벅스_매장수'})
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수
0,11320,도봉구,37.665861,127.031767,2
1,11380,은평구,37.617612,126.922700,8
2,11230,동대문구,37.583801,127.050700,8
3,11590,동작구,37.496504,126.944307,11
4,11545,금천구,37.460097,126.900155,11


In [282]:
# seoul_sgg에 '종사자수', '사업체수' 칼럼을 추가하시오..
seoul_sgg_biz = pd.read_excel('./data/4_3_sgg_biz.xlsx')
seoul_sgg_biz.head()

,시군구명,종사자수,사업체수
0,종로구,269106,40871
1,중구,423808,66190
2,용산구,133446,21178
3,성동구,162019,26130
4,광진구,123689,24531


In [283]:
seoul_sgg = pd.merge(seoul_sgg, seoul_sgg_biz, on='시군구명')
seoul_sgg.head()

,시군구코드,시군구명,위도,경도,스타벅스_매장수,종사자수,사업체수
0,11320,도봉구,37.665861,127.031767,2,68669,18455
1,11380,은평구,37.617612,126.922700,8,87693,24179
2,11230,동대문구,37.583801,127.050700,8,143858,32994
3,11590,동작구,37.496504,126.944307,11,103915,19609
4,11545,금천구,37.460097,126.900155,11,223058,30080


In [284]:
seoul_sgg.to_excel('./data/4_5_seoul_sgg_stat.xlsx', index=False)